# Genetic Linear Regression with DuckDB
A solution... via evolution!

(Testing my skillz. Can I do this in SQL?)

In [1]:
import duckdb as db

In [2]:
observations = db.sql('''
WITH RECURSIVE numbers AS (
    -- Start with 0
    SELECT 0 AS num
    UNION ALL
    -- Add 1 to num until we reach 9 (10 rows)
    SELECT num + 1 FROM numbers WHERE num < 9
), 
parameters AS (
    SELECT 
        5 AS mean, 
        2.0 AS stddev  
),
random AS (
    SELECT
    (sqrt(-2 * ln(random())) * cos(2 * pi() * random()) * stddev + mean) AS norm_rand
    FROM numbers, parameters
)
SELECT 
    norm_rand as x,
    3.0 AS b0_true, 
    5.0 AS b1_true,
    b0_true + b1_true*x AS y
FROM random;
''') 
observations

┌────────────────────┬──────────────┬──────────────┬────────────────────┐
│         x          │   b0_true    │   b1_true    │         y          │
│       double       │ decimal(2,1) │ decimal(2,1) │       double       │
├────────────────────┼──────────────┼──────────────┼────────────────────┤
│  4.953917635584684 │          3.0 │          5.0 │  27.76958817792342 │
│  5.310234260717173 │          3.0 │          5.0 │ 29.551171303585864 │
│   4.86398646412988 │          3.0 │          5.0 │   27.3199323206494 │
│  6.655820379113392 │          3.0 │          5.0 │  36.27910189556696 │
│ 1.0336996892600254 │          3.0 │          5.0 │  8.168498446300127 │
│  4.289592467691411 │          3.0 │          5.0 │ 24.447962338457053 │
│  4.207910905235669 │          3.0 │          5.0 │ 24.039554526178343 │
│  7.307505907859962 │          3.0 │          5.0 │  39.53752953929981 │
│  5.682932976341023 │          3.0 │          5.0 │ 31.414664881705114 │
│ 2.9620652744915454 │          3.0 │ 

In [14]:
betas = db.sql('''
WITH RECURSIVE numbers AS (
    SELECT 0 AS num
    UNION ALL
    SELECT num + 1 FROM numbers WHERE num < 9
), 
betas as (
SELECT 
    random()*20 - 10 AS b0,
    random()*20 - 10 AS b1
    FROM numbers
    )
SELECT b0, b1, 1 AS generation
FROM betas; 
''') 
betas

┌─────────────────────┬─────────────────────┬────────────┐
│         b0          │         b1          │ generation │
│       double        │       double        │   int32    │
├─────────────────────┼─────────────────────┼────────────┤
│ -0.8783239684998989 │   8.467420260421932 │          1 │
│   9.601055639795959 │  0.2372026862576604 │          1 │
│  0.3898862097412348 │   -2.06641492433846 │          1 │
│   8.587141833268106 │  -7.939202627167106 │          1 │
│  -8.962439172901213 │   4.019545568153262 │          1 │
│  -8.358679409138858 │  -7.355219479650259 │          1 │
│   8.263101475313306 │ -6.8068417347967625 │          1 │
│ -3.4639610769227147 │ -2.8209835197776556 │          1 │
│  -7.126688379794359 │  -7.238796618767083 │          1 │
│ -2.8277820348739624 │    9.47117280215025 │          1 │
├─────────────────────┴─────────────────────┴────────────┤
│ 10 rows                                      3 columns │
└───────────────────────────────────────────────────────

In [23]:
db.sql('''
WITH 
max_gen as 
     (SELECT MAX(generation) AS generation FROM betas),
last_betas as (
    select * from betas
    where generation = (SELECT generation FROM max_gen)
 ),
 
cp AS
    (SELECT o.*,
            b.b0,
            b.b1,
            b.generation,
            ((b.b0 + b.b1 * o.x) - o.y) ^ 2 AS sq_error
     FROM observations as o
     CROSS JOIN last_betas as b),
     
rmse_calc AS
    (SELECT b0, b1, generation, SQRT(AVG(sq_error)) AS rmse
     FROM cp
     GROUP BY b0, b1, generation),
     
ranked as
    (SELECT *, rank() over(partition by generation order by rmse) as rank
     FROM rmse_calc )
     
SELECT *, (SELECT generation FROM max_gen) as maxgen
FROM ranked;
''') 

┌─────────────────────┬─────────────────────┬────────────┬────────────────────┬───────┬────────┐
│         b0          │         b1          │ generation │        rmse        │ rank  │ maxgen │
│       double        │       double        │   int32    │       double       │ int64 │ int32  │
├─────────────────────┼─────────────────────┼────────────┼────────────────────┼───────┼────────┤
│ -1.8383916281163692 │    8.29119781497866 │          1 │ 11.763730612432596 │     1 │      1 │
│ -1.1223242152482271 │   2.498762747272849 │          1 │ 16.416061604163776 │     2 │      1 │
│   7.891862401738763 │ -0.7992114126682281 │          1 │ 24.122727187405637 │     3 │      1 │
│ 0.41417579632252455 │ 0.48008312471210957 │          1 │ 24.930193019588348 │     4 │      1 │
│   2.784040803089738 │ -3.5660387482494116 │          1 │  42.72625430748721 │     5 │      1 │
│  6.2269825814291835 │  -6.202524434775114 │          1 │  52.51209925191693 │     6 │      1 │
│    7.19915057066828 │ -7.988

In [22]:
db.sql('''
select * from tabl
''') 

┌─────────────────────┬─────────────────────┬────────────┬────────────────────┬───────┬────────┐
│         b0          │         b1          │ generation │        rmse        │ rank  │ maxgen │
│       double        │       double        │   int32    │       double       │ int64 │ int32  │
├─────────────────────┼─────────────────────┼────────────┼────────────────────┼───────┼────────┤
│ -2.8421303629875183 │   7.103150780312717 │          1 │  7.123641660035427 │     1 │      1 │
│    3.19977268576622 │  3.3761978335678577 │          1 │  9.121425419936074 │     2 │      1 │
│  -5.655125980265439 │   8.512225593440235 │          1 │  12.71705080005476 │     3 │      1 │
│ -0.7297514192759991 │   3.382140463218093 │          1 │ 12.766491425465453 │     4 │      1 │
│   1.178553937934339 │    9.20008392073214 │          1 │  22.41300853352538 │     5 │      1 │
│  3.3771767700091004 │  0.6373874330893159 │          1 │ 24.651428471320123 │     6 │      1 │
│  -1.746843452565372 │  1.318

In [ ]:
observations = db.sql('''

''') 